In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from config import props, filenames
import analysis
from scipy.signal import find_peaks

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
SAVE_FIGS = False

In [ ]:
data_raw = pd.read_pickle(filenames.data+'data.pkl')
data_raw.t = data_raw.t - data_raw.t[0]
data_convoluted = analysis.data_cleaning(data_raw.copy())
pert_times = analysis.find_perts(data_convoluted.copy())
data = analysis.data_interpolation(data_convoluted.copy(), pert_times)

In [ ]:
data = analysis.data_interpolation(data_convoluted.copy(), pert_times)

In [ ]:
plt.plot(np.diff(data_convoluted.I, n=2, prepend=0, append=0))

In [ ]:
# mpl.use('pgf')
cut_data = data_convoluted[(data_convoluted.t>8250) & (data_convoluted.t < 8400)]
fig = plt.figure(figsize=[3.2,2.4])
plt.plot(cut_data.t, cut_data.I)
plt.title('High Amplitude oscillation, 6V')
plt.xlabel('Time [s]')
plt.ylabel(r'J [mA/cm$^2$]')
plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)
plt.show()

In [ ]:
plt.figure(figsize=[3.2,2.4])
plt.plot(cut_data.I, cut_data.emsi)
plt.xlabel(r'J [mA/cm$^2$]')
plt.ylabel(r'$\xi$ [\%]')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(data_raw.t, data_raw.I, label='raw')
plt.scatter(pert_times, data_raw.loc[data_raw.t.isin(pert_times), 'I'], marker='x', c='r')
plt.plot(data.t, data.I, label='treated')
plt.legend(loc=1)
plt.show()
if SAVE_FIGS:
    plt.savefig(filenames.notes+'current_waveform.png')

In [ ]:
cycles = analysis.find_cycles(data, pert_times)

In [ ]:
perts = analysis.pert_response(data, cycles, pert_times[:-1])

In [ ]:
fig, axs = plt.subplots(2, sharex=True)

axs[0].plot(cycles['start'], cycles['duration'])
axs[0].plot(perts.time, perts.expected_period)
axs[0].set_ylabel('period')
axs[0].scatter(cycles['start'][cycles['had_pert']], cycles['duration'][cycles['had_pert']], c='r', s=10)

axs[1].scatter(perts['time'], perts['phase'], marker = 'x')
# axs[1].plot(perts['time'], perts['phase'])
axs[1].set_ylabel('pert. phase')

fig.supxlabel('time [s]')
fig.tight_layout()
if SAVE_FIGS:
    fig.savefig(filenames.notes+'period_vs_time.png')

In [ ]:
def on_pick(event) -> tuple[mpl.figure.Figure, mpl.figure.Axes]:
    ax_phase.cla()
    ax_current.cla()

    pert_time = perts.time.iloc[event.ind[0]]
    print(pert_time)
    data_before = data_convoluted[(data.t > pert_time - props.max_period) & (data.t < pert_time)]
    data_after = data_convoluted[(data.t > pert_time) & (data.t < pert_time + 2*props.max_period)]

    ax_phase.plot(data_before.I, data_before.emsi)
    ax_phase.plot(data_after.I, data_after.emsi)
    ax_phase.scatter(data_before.I.iloc[-1], data_before.emsi.iloc[-1], c='r')
    fig_phase.suptitle(rf'Perturbation at $\phi$ = {perts.phase.iloc[event.ind[0]]:.2f} -- phase space')
    fig_phase.supxlabel(r'current [mA/cm$^2$]')
    fig_phase.supylabel(r'emsi signal [$\xi$]')
    fig_phase.canvas.draw()

    ax_current.plot(data_before.t, data_before.I)
    ax_current.plot(data_after.t, data_after.I)
    ax_current.scatter(data_before.t.iloc[-1], data_before.I.iloc[-1], c='r')
    fig_current.suptitle(rf'Perturbation at $\phi$ = {perts.phase.iloc[event.ind[0]]:.2f} -- current')
    fig_current.supxlabel('time [s]')
    fig_current.supylabel(r'current [mA/cm$^2$]')
    fig_current.canvas.draw()

In [ ]:
if props.pert_type == 'U':
    title = f'PRC for {props.osc_type} {props.voltage}{'+' if props.pert_strength>0 else '-'}{props.pert_strength}V, {props.pert_dt}s'
else:
    title = f'PRC for {props.voltage}, {props.pert_dt}s {'+' if props.pert_strength>0 else '-'}ve light'
fig, ax = plt.subplots()
fig.suptitle(title)
ax.axhline(0)
ax.scatter(perts['phase'], perts['response'], c=perts['time'], picker=True)
fig_phase, ax_phase = plt.subplots()
fig_current, ax_current = plt.subplots()
id = fig.canvas.mpl_connect('pick_event', on_pick)
# if SAVE_figS:
#     fig.save_fig(filenames.notes+'prc.png')

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
fig.suptitle(title)
axs[0].axhline(0)
axs[0].scatter(perts['phase'], perts['response'], c=perts['time'])
axs[0].set_ylabel('Phase Response')
one_period = data_convoluted[(data_convoluted.t>cycles.loc[100, 'start']) & (data_convoluted.t<cycles.loc[101, 'start'])]
axs[1].plot((one_period.t-cycles.start[100])/cycles.loc[100, 'duration'], one_period.I/props.el_area*100)
axs[1].set_ylabel(r'J [mA/cm$^2$]')
plt.show()

In [ ]:
fig.canvas.mpl_disconnect(14)

In [ ]:
id = fig.canvas.mpl_connect('pick_event', on_pick)

In [ ]:
fig, axs = plt.subplots(2,2, sharex = True, sharey=True, figsize = (14, 9))
things_to_plot = ['response_0', 'response_1', 'response_2', 'response_3']
for (thing, ax) in zip(things_to_plot, axs.flatten()):
    ax.scatter(perts['phase'], perts[thing], c=perts['time'])
    ax.set_title(thing)
    ax.axhline(0, ls='--')
fig.suptitle(title)
fig.supxlabel(r'$\phi$')
fig.supylabel(r'$\Delta\phi$')
# fig.tight_layout(pad=0.5, w_pad=0.5, h_pad=0.5)
fig.tight_layout()
if SAVE_FIGS:
    fig.savefig(filenames.notes+'prc_full.png')